<a href="https://colab.research.google.com/github/NgoHoangNhatMinh/SUTD-Hackathon/blob/main/Game_Of_Life_Julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
versioninfo()

In [ ]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

In [ ]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

In [ ]:
function ca_evolve(init_vec, nt, rule)
    nl = size(init_vec,1);
    system = zeros(Int, nl, nt);
    system[:, 1] = init_vec;
    #evolution
    for it = 2:nt
        for il = 2:nl-1
            system[il,it] = rule_function(system[il-1:il+1,it-1], rule);
        end
        #boundary conditions
        system[1, it] = rule_function([system[nl, it-1], system[1, it-1], system[2, it-1]],rule);
        system[nl, it] = rule_function([system[nl-1, it-1], system[nl, it-1], system[1, it-1]],rule);
    end
    return system
end

function rule_function(vec, rule)
    if rule == 30
        if vec == [0, 0, 0]
            return 0
        elseif vec == [0, 0, 1]
            return 1
        elseif vec == [0, 1, 0]
            return 1
        elseif vec == [0, 1, 1]
            return 1
        elseif vec == [1, 0, 0]
            return 1
        elseif vec == [1, 0, 1]
            return 0
        elseif vec == [1, 1, 0]
            return 0
        else
            return 0
        end
    elseif rule == 18
        if vec == [0, 0, 0]
            return 0
        elseif vec == [0, 0, 1]
            return 1
        elseif vec == [0, 1, 0]
            return 0
        elseif vec == [0, 1, 1]
            return 0
        elseif vec == [1, 0, 0]
            return 1
        elseif vec == [1, 0, 1]
            return 0
        elseif vec == [1, 1, 0]
            return 0
        else
            return 0
        end
    else #rule 3
        if vec == [0, 0, 0]
            return 1
        elseif vec == [0, 0, 1]
            return 1
        elseif vec == [0, 1, 0]
            return 0
        elseif vec == [0, 1, 1]
            return 0
        elseif vec == [1, 0, 0]
            return 0
        elseif vec == [1, 0, 1]
            return 0
        elseif vec == [1, 1, 0]
            return 0
        else
            return 0
        end
    end
end

In [ ]:
L = 201; #size of system, odd
nt = 300; #time steps

init_vec = zeros(Int, L);
init_vec[div(L+1,2)] = 1;

rule = 30;

system = ca_evolve(init_vec, nt, rule);


In [ ]:
using Plots
heatmap(system; axis = nothing, border = :none, cbar = false)